In [ ]:
# Install these packages in your Python environment (not in Power BI itself):
# pip install google-api-python-client google-auth-httplib2 google-auth-oauthlib openpyxl pandas

import pandas as pd
import io
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload

# --- CONFIGURATION ---
SERVICE_ACCOUNT_FILE = r'/content/finance-460901-67416d83fb09.json'  # Update this path
SCOPES = ['https://www.googleapis.com/auth/drive.readonly']
FOLDER_ID = '1I7iIQJiF_tV9AQaqa9lx-6Xz_rVaBIET'  # Update with your Google Drive folder ID

# --- AUTHENTICATION ---
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=credentials)

# --- LIST FILES IN FOLDER ---
def list_files(service, folder_id):
    files = []
    page_token = None
    while True:
        response = service.files().list(
            q=f"'{folder_id}' in parents and trashed=false",
            fields="nextPageToken, files(id, name, mimeType)",
            pageToken=page_token
        ).execute()
        files.extend(response.get('files', []))
        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break
    return files

files = list_files(service, FOLDER_ID)

# --- DOWNLOAD AND READ FILES ---
def download_drive_file(service, file_id):
    request = service.files().get_media(fileId=file_id)
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while not done:
        status, done = downloader.next_chunk()
    fh.seek(0)
    return fh

file_dataframes = []
for file in files:
    file_id = file['id']
    file_name = file['name']
    mime_type = file['mimeType']
    try:
        if mime_type == 'application/vnd.google-apps.spreadsheet':
            # Export Google Sheet as Excel (xlsx)
            request = service.files().export_media(
                fileId=file_id,
                mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet'
            )
            fh = io.BytesIO()
            downloader = MediaIoBaseDownload(fh, request)
            done = False
            while not done:
                status, done = downloader.next_chunk()
            fh.seek(0)
            df = pd.read_excel(fh, engine='openpyxl')
            df['SourceFile'] = file_name
            file_dataframes.append(df)
        elif mime_type == 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet':
            fh = download_drive_file(service, file_id)
            fh.seek(0)
            df = pd.read_excel(fh, engine='openpyxl')
            df['SourceFile'] = file_name
            file_dataframes.append(df)
        elif mime_type == 'text/csv':
            fh = download_drive_file(service, file_id)
            df = pd.read_csv(fh)
            df['SourceFile'] = file_name
            file_dataframes.append(df)
        else:
            # Skip unsupported file types
            continue
    except Exception as e:
        # Optionally, log or handle file read errors
        continue

# --- COMBINE ALL DATAFRAMES ---
if file_dataframes:
    combined_df = pd.concat(file_dataframes, ignore_index=True)
else:
    combined_df = pd.DataFrame()

# --- OUTPUT FOR POWER BI ---
combined_df


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,...,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score,SourceFile
0,0x1bbea,CUS_0x2863,September,Serapiow,46,150-77-6507,Architect,80011.64,6824.636667,7,...,792.92,33.147225,5 Years and 11 Months,Yes,20133.000000,144.32152591037604,High_spent_Large_value_payments,623.580527109982,NaN,combined_part_7.xlsx
1,0x1bbeb,CUS_0x2863,October,Serapiow,46,150-77-6507,Architect,80011.64_,6824.636667,7,...,792.92,29.278855,6 Years and 0 Months,Yes,154.561614,__10000__,!@9#%8,622.875977954279,NaN,combined_part_7.xlsx
2,0x1bbec,CUS_0x2863,November,Serapiow,46,150-77-6507,Architect,80011.64,NaN,7,...,792.92,39.316822,6 Years and 1 Months,Yes,154.561614,139.41638078622734,High_spent_Large_value_payments,628.4856722341308,NaN,combined_part_7.xlsx
3,0x1bbed,CUS_0x2863,December,Serapiow,3277,150-77-6507,Architect,80011.64,NaN,7,...,792.92,38.469938,6 Years and 2 Months,Yes,154.561614,573.9865627619063,Low_spent_Medium_value_payments,233.91549025845188,NaN,combined_part_7.xlsx
4,0x1bbf6,CUS_0xc4b6,September,NaN,33,027-26-6337,Musician,119491.24,9782.603333,1,...,735.75,30.177254,17 Years and 7 Months,No,118.060785,111.60239773434185,High_spent_Large_value_payments,988.5971510809571,NaN,combined_part_7.xlsx
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,0x1bbd5,CUS_0x9b20,December,Raymondr,36,746-63-3604,Entrepreneur,28033.88,2571.156667,8,...,3719.49,28.401527,3 Years and 2 Months,Yes,100.708062,96.9598762050703,Low_spent_Small_value_payments,349.447729,NaN,combined_part_6.xlsx
11996,0x1bbde,CUS_0x5344,September,Lucia Mutikanig,51,566-56-7727,_______,22330.07,1755.839167,1,...,470.36,40.163258,19 Years and 10 Months,No,11.709562,19.289882179919992,High_spent_Large_value_payments,384.584473,NaN,combined_part_6.xlsx
11997,0x1bbdf,CUS_0x5344,October,NaN,51,#F%$D@*&8,Engineer,22330.07,1755.839167,1,...,470.36,25.428585,19 Years and 11 Months,No,11.709562,161.9050872594124,Low_spent_Small_value_payments,291.969268,NaN,combined_part_6.xlsx
11998,0x1bbe0,CUS_0x5344,November,Lucia Mutikanig,51,566-56-7727,Engineer,22330.07,1755.839167,1,...,470.36,37.531276,20 Years and 0 Months,No,11.709562,31.882750126917106,High_spent_Medium_value_payments,381.991605,NaN,combined_part_6.xlsx
